In [1]:
%reset
import numpy as np 
import pandas as pd 
import xarray as xr 
import cartopy 
import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import climlab.utils.thermo as climlab
import scipy 
import datetime
import metpy.calc as mpcalc
import netCDF4

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Load Data

In [2]:
root_path='/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/'
kernel_path='/data/keeling/a/rytam2/a/kernels/gcms/'

# Process Data
For each model, 
1. Select all ta/ts files 
2. Extract t700 from ta files for EIS calculation and ts from ts files for SST
3. Adjust data lon coordinates from 0:360 to -180:180 (to match kernel coordinates) 
4. Transpose dimensions for interp_like method to work
5. Extract coordinates from Kernels 
6. Interpolate the grid from 1x1 to 5x5 with interp_like 
7. Adjust time coordinates for merging 

In [5]:
modelfilename_list = ['CCSM4','CanESM2','CanESM5','E3SM','GFDL','HadGEM2','HadGEM3','IPSL','MIROC-ES2L','MIROC-ESM',
                     'MIROC5','MIROC6','MPI-ESM','MRI-CGCM3','MRI-ESM2','UKESM']

modelname_list=['ccsm4','canam4','canesm5','e3sm','gfdl','hg2','hg3','ipsl','mies2l','miesm','mi5','mi6','mpi','mrcgcm','mresm','ukesm']

cmip5modelname_list = ['ccsm4','canam4','hg2','miesm','mpi','mrcgcm'] #except miroc5 

modelvar_list = ['ta','ts','hur','sfcWind','wap','ua','va']

dict_ds_t700 = {}
dict_ds_ts = {}
dict_ds_hur700 = {}
dict_ds_sfcWind = {}
dict_ds_uas = {}
dict_ds_vas = {}
dict_ds_wap = {}
dict_ds_tadv = {}

# Get datetime64 time coordinates for missing NaN array 
time_ds=xr.open_mfdataset('/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/wap_Amon_MIROC-ES2L_historical_r1i1p1f2_gn_185001-201412.nc')
index_nan=time_ds.indexes['time'][155*12:159*12]

#interp grids (selected CCSM4 for coordinates)
path = kernel_path+'*CCSM4*.nc'
kernel = xr.open_mfdataset(path)

for modelname in modelname_list:

    for var in modelvar_list:
        # Get all files 
        filepath = glob.glob(root_path+'%s*%s*.nc'%(var,modelfilename_list[modelname_list.index(modelname)]))
        print(modelname,len(filepath),'\n',filepath,'\n')
        
        # If-loop Combine all .nc files as one dataset 
        if len(filepath)==1:
            ds = xr.open_mfdataset(filepath);
        elif len(filepath) > 1: 
            ds = xr.open_mfdataset(filepath,combine="by_coords");

        # Select variable from dataset 
        if var == 'ta':
            ds = ds.ta.sel(plev=70000) #select 700hPa 
        elif var == 'ts':
            ds = ds.ts
        elif var == 'hur':
            ds = ds.hur.sel(plev=70000)
        elif var == 'sfcWind':
            if modelname == 'ccsm4': #calculate ccsm4 WS with ua/va 
                pass
            else:
                ds = ds.sfcWind
        elif var == 'ua':
            if modelname == 'ccsm4' or modelname == 'e3sm':
                ds = ds.ua.sel(plev=1e5)
            else: 
                ds = ds.uas
        elif var == 'va':
            if modelname == 'ccsm4' or modelname =='e3sm':
                ds = ds.va.sel(plev=1e5)
            else: 
                ds = ds.vas
        elif var == 'wap': 
            ds = ds.wap.sel(plev=70000)
            

        # Change lon coords
        ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
        ds.lat.attrs['units'] = 'degrees' 
        ds.lon.attrs['units'] = 'degrees' 

      
        # Unify time coords to start in year 1982 and end in 2008
        if modelname == 'hg2':
            ds = ds.isel(time=slice(122*12+1,-1)) #198201- 200510
        else:    
            ds = ds.isel(time=slice(132*12,159*12))
            print(np.shape(ds))
            
        
        # Unify time coordinates as datetime64
        if len(filepath)==0: 
            pass 
        elif len(filepath)!=0 :
            if modelname == 'hg2':
                index=time_ds.indexes['time'][132*12:156*12-2] #hg2
                print(index[0], index[-1])
            elif modelname in cmip5modelname_list and modelname != 'hg2':
                index=time_ds.indexes['time'][132*12:156*12] #cmip5
                print(index[0], index[-1])
            else:
                index=time_ds.indexes['time'][132*12:159*12] #cmip6
                print(index[0], index[-1])
            ds['time']=index
            
        print(modelname, np.shape(ds))
        
        #interp grids (selected CCSM4 for coordinates)
        kernel_ds = kernel.dRdxi.isel(i=0).expand_dims({'time':time_ds.indexes['time'][132*12:159*12]}, axis=2)\
            .assign_coords({'latitude':kernel.lat,'longitude':kernel.lon})\
            .rename({'latitude':'lat','longitude':'lon'})

            
        
        # Interp variables and assign to dict 
        if var == 'ta':
            # interp ts
            ds = ds.interp_like(kernel_ds)
            dict_ds_t700[modelname] = ds
        elif var == 'ts':
            ds = ds.interp_like(kernel_ds)
            dict_ds_ts[modelname] = ds
        elif var == 'hur':
            ds = ds.interp_like(kernel_ds)
            dict_ds_hur700[modelname] = ds
        elif var == 'sfcWind':
            ds = ds.interp_like(kernel_ds)
            dict_ds_sfcWind[modelname] = ds
        elif var == 'ua':
            ds = ds.interp_like(kernel_ds)
            dict_ds_uas[modelname] = ds
        elif var == 'va':
            ds = ds.interp_like(kernel_ds)
            dict_ds_vas[modelname] = ds
        elif var == 'wap':
            ds = ds.interp_like(kernel_ds)
            dict_ds_wap[modelname] = ds
            
        
    ds = mpcalc.advection(dict_ds_ts[modelname],u=dict_ds_uas[modelname], v=dict_ds_vas[modelname]) 
    dict_ds_tadv[modelname] = ds
    
        
# Change dictionaries to Xarray
ts = xr.Dataset(dict_ds_ts)
hur = xr.Dataset(dict_ds_hur700)
ws = xr.Dataset(dict_ds_sfcWind)
wap = xr.Dataset(dict_ds_wap)
tadv = xr.Dataset(dict_ds_tadv)
t700 = xr.Dataset(dict_ds_t700)
uas = xr.Dataset(dict_ds_uas)
vas = xr.Dataset(dict_ds_vas)


ccsm4 3 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_CCSM4_historical_r1i1p1_185001-189912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_CCSM4_historical_r1i1p1_190001-194912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_CCSM4_historical_r1i1p1_195001-200512.nc'] 

(288, 192, 288)
1982-01-16 12:00:00 2005-12-16 12:00:00
ccsm4 (288, 192, 288)
ccsm4 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ts_Amon_CCSM4_historical_r1i1p1_185001-200512.nc'] 

(288, 192, 288)
1982-01-16 12:00:00 2005-12-16 12:00:00
ccsm4 (288, 192, 288)
ccsm4 3 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_CCSM4_historical_r1i1p1_185001-189912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_CCSM4_historical_r1i1p1_190001-194912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_CCSM4_historical_r1i1p1_195001-200512.nc'] 

(288, 192, 288)
19

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/metpy/xarray.py:1473: UserWarning: Vertical dimension number not found. Defaulting to (..., Z, Y, X) order.
  warnings.warn(


canam4 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_CanESM2_historical_r1i1p1_185001-200512.nc'] 

(288, 64, 128)
1982-01-16 12:00:00 2005-12-16 12:00:00
canam4 (288, 64, 128)
canam4 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ts_Amon_CanESM2_historical_r1i1p1_185001-200512.nc'] 

(288, 64, 128)
1982-01-16 12:00:00 2005-12-16 12:00:00
canam4 (288, 64, 128)
canam4 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_CanESM2_historical_r1i1p1_185001-200512.nc'] 

(288, 64, 128)
1982-01-16 12:00:00 2005-12-16 12:00:00
canam4 (288, 64, 128)
canam4 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/sfcWind_Amon_CanESM2_historical_r1i1p1_185001-200512.nc'] 

(288, 64, 128)
1982-01-16 12:00:00 2005-12-16 12:00:00
canam4 (288, 64, 128)
canam4 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/wap_Amon_CanESM2_historical_r1i1p1_185001-200512.nc'] 

(288, 64, 128)
1982-01-16 12:00:00 2005-12-16

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/metpy/xarray.py:1473: UserWarning: Vertical dimension number not found. Defaulting to (..., Z, Y, X) order.
  warnings.warn(


canesm5 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_CanESM5_historical_r1i1p1f1_gn_185001-201412.nc'] 

(324, 64, 128)
1982-01-16 12:00:00 2008-12-16 12:00:00
canesm5 (324, 64, 128)
canesm5 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ts_Amon_CanESM5_historical_r1i1p1f1_gn_185001-201412.nc'] 

(324, 64, 128)
1982-01-16 12:00:00 2008-12-16 12:00:00
canesm5 (324, 64, 128)
canesm5 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_CanESM5_historical_r1i1p1f1_gn_185001-201412.nc'] 

(324, 64, 128)
1982-01-16 12:00:00 2008-12-16 12:00:00
canesm5 (324, 64, 128)
canesm5 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/sfcWind_Amon_CanESM5_historical_r1i1p1f1_gn_185001-201412.nc'] 

(324, 64, 128)
1982-01-16 12:00:00 2008-12-16 12:00:00
canesm5 (324, 64, 128)
canesm5 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/wap_Amon_CanESM5_historical_r1i1p1f1_gn_185001-201412.nc'] 

(324, 64, 1

...

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/metpy/xarray.py:1473: UserWarning: Vertical dimension number not found. Defaulting to (..., Z, Y, X) order.
  warnings.warn(


mresm 4 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_185001-189912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_190001-194912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_195001-199912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_200001-201412.nc'] 

(324, 160, 320)
1982-01-16 12:00:00 2008-12-16 12:00:00
mresm (324, 160, 320)
mresm 1 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ts_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_185001-201412.nc'] 

(324, 160, 320)
1982-01-16 12:00:00 2008-12-16 12:00:00
mresm (324, 160, 320)
mresm 4 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_185001-189912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_MRI-

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/metpy/xarray.py:1473: UserWarning: Vertical dimension number not found. Defaulting to (..., Z, Y, X) order.
  warnings.warn(


ukesm 2 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_185001-194912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ta_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_195001-201412.nc'] 

(324, 144, 192)
1982-01-16 12:00:00 2008-12-16 12:00:00
ukesm (324, 144, 192)
ukesm 2 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ts_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_185001-194912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/ts_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_195001-201412.nc'] 

(324, 144, 192)
1982-01-16 12:00:00 2008-12-16 12:00:00
ukesm (324, 144, 192)
ukesm 2 
 ['/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_185001-194912.nc', '/data/keeling/a/rytam2/ccf_model_spread/data/raw/historical/hur_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_195001-201412.nc'] 

(324, 144, 192)
1982-01-16 12:00:00 2008-12-16 12:00:00
ukesm (32

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/metpy/xarray.py:1473: UserWarning: Vertical dimension number not found. Defaulting to (..., Z, Y, X) order.
  warnings.warn(


In [6]:
# Calcualte EIS 
eis = climlab.EIS(ts,t700)

# Calculate ws from ua/va for CCSM4 
#Metpy has Pint quantity units; chunk removes that 
ws['ccsm4'] = mpcalc.wind_speed(uas.ccsm4,vas.ccsm4).rename('ccsm4').chunk()
ws=ws.drop(['plev','height'])

# Change tadv to dask 
tadv=tadv.chunk()

In [7]:
# Save as nc files 
path='/data/keeling/a/rytam2/ccf_model_spread/data/preprocessed/'

ts.to_netcdf(path+'ts_obshist_CMIP5&6_2008.nc')
eis.to_netcdf(path+'eis_obshist_CMIP5&6_2008.nc')
hur.drop('plev').to_netcdf(path+'hur_obshist_CMIP5&6_2008.nc')
ws.to_netcdf(path+'ws_obshist_CMIP5&6_2008.nc')
wap.drop('plev').to_netcdf(path+'wap_obshist_CMIP5&6_2008.nc')
tadv.to_netcdf(path+'tadv_obshist_CMIP5&6_2008.nc')

## Comments

### Improvements: 
1. Try using dask to stack? 